# 问题二：单词属性与困难模式占比的关联分析

## 任务要求
1. 判断目标单词的属性（字母频率、重复字母数、元音数量等）是否影响「困难模式玩家占比」
2. 若有影响，需量化说明
3. 若无影响，需解释原因

## 建模思路
1. **相关性分析**：计算单词属性与困难模式占比的Pearson/Spearman相关系数
2. **多元线性回归**：分析各属性的回归系数和显著性（p值）
3. **随机森林**：评估特征重要性（非线性关系）
4. **结论与解释**：基于统计证据得出结论

---
## 1. 数据加载与准备

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# 设置绑图标准配置
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style='whitegrid')

# 标准尺寸与配色
FIGSIZE_WIDE = (12, 6)
FIGSIZE_NORMAL = (10, 6)
FIGSIZE_SQUARE = (8, 8)
COLORS = {
    'primary': '#4682B4',
    'secondary': '#FF7F50',
    'accent': '#228B22',
    'neutral': '#708090'
}

print('库导入完成')

In [ ]:
# 加载预处理后的数据
df = pd.read_csv('../数据预处理/data_processed.csv')
df['date'] = pd.to_datetime(df['date'])

print(f'数据加载成功')
print(f'数据形状: {df.shape}')
print(f'\n困难模式占比统计:')
print(df['hard_mode_ratio'].describe())

In [ ]:
# 定义分析变量
# 目标变量：困难模式占比
target = 'hard_mode_ratio'

# 单词属性特征
word_features = [
    'num_vowels',           # 元音数量
    'vowel_ratio',          # 元音占比
    'num_unique_letters',   # 不重复字母数
    'num_repeated_letters', # 重复字母数
    'has_repeated',         # 是否有重复字母
    'avg_letter_freq',      # 平均字母频率
    'min_letter_freq',      # 最小字母频率
    'max_letter_freq',      # 最大字母频率
    'first_letter_freq',    # 首字母频率
    'last_letter_freq'      # 尾字母频率
]

print(f'目标变量: {target}')
print(f'单词属性特征 ({len(word_features)}个): {word_features}')

---
## 2. 探索性分析

### 2.1 困难模式占比分布

**图1说明**：困难模式占比的分布直方图，了解目标变量的分布特征。

In [ ]:
# 图1: 困难模式占比分布
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 直方图
axes[0].hist(df['hard_mode_ratio'] * 100, bins=30, color=COLORS['primary'], 
             edgecolor='black', alpha=0.7)
axes[0].axvline(x=df['hard_mode_ratio'].mean() * 100, color='red', 
                linestyle='--', linewidth=2, label=f'Mean: {df["hard_mode_ratio"].mean()*100:.2f}%')
axes[0].set_xlabel('Hard Mode Ratio (%)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].legend()

# 时间序列
df_sorted = df.sort_values('date')
axes[1].plot(df_sorted['date'], df_sorted['hard_mode_ratio'] * 100, 
             color=COLORS['primary'], alpha=0.6, linewidth=1)
ma7 = df_sorted['hard_mode_ratio'].rolling(7).mean() * 100
axes[1].plot(df_sorted['date'], ma7, color=COLORS['secondary'], linewidth=2, label='7-day MA')
axes[1].set_xlabel('Date', fontsize=12)
axes[1].set_ylabel('Hard Mode Ratio (%)', fontsize=12)
axes[1].legend()

plt.tight_layout()
plt.savefig('figures/fig1_hard_mode_distribution.pdf', bbox_inches='tight')
plt.show()
print('图1已保存: figures/fig1_hard_mode_distribution.pdf')

In [ ]:
# 困难模式占比统计
print('困难模式占比统计:')
print(f'  均值: {df["hard_mode_ratio"].mean()*100:.2f}%')
print(f'  标准差: {df["hard_mode_ratio"].std()*100:.2f}%')
print(f'  最小值: {df["hard_mode_ratio"].min()*100:.2f}%')
print(f'  最大值: {df["hard_mode_ratio"].max()*100:.2f}%')
print(f'  变异系数: {df["hard_mode_ratio"].std()/df["hard_mode_ratio"].mean()*100:.2f}%')

### 2.2 单词属性分布

**图2说明**：各单词属性特征的分布情况。

In [ ]:
# 图2: 单词属性分布
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

plot_features = ['num_vowels', 'num_repeated_letters', 'avg_letter_freq', 
                 'min_letter_freq', 'num_unique_letters', 'vowel_ratio']

for i, feat in enumerate(plot_features):
    axes[i].hist(df[feat], bins=20, color=COLORS['primary'], edgecolor='black', alpha=0.7)
    axes[i].axvline(x=df[feat].mean(), color='red', linestyle='--', linewidth=2)
    axes[i].set_xlabel(feat, fontsize=11)
    axes[i].set_ylabel('Frequency', fontsize=11)

plt.tight_layout()
plt.savefig('figures/fig2_word_features_distribution.pdf', bbox_inches='tight')
plt.show()
print('图2已保存: figures/fig2_word_features_distribution.pdf')

---
## 3. 相关性分析

### 3.1 Pearson相关系数

**图3说明**：单词属性与困难模式占比的相关性热力图。

In [ ]:
# 计算相关系数
corr_features = word_features + [target]
corr_matrix = df[corr_features].corr()

# 图3: 相关性热力图
fig, ax = plt.subplots(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='RdBu_r', center=0, 
            fmt='.3f', square=True, linewidths=0.5, ax=ax,
            cbar_kws={'shrink': 0.8})
plt.tight_layout()
plt.savefig('figures/fig3_correlation_heatmap.pdf', bbox_inches='tight')
plt.show()
print('图3已保存: figures/fig3_correlation_heatmap.pdf')

In [ ]:
# 提取与困难模式占比的相关系数
correlations = []
for feat in word_features:
    r, p = stats.pearsonr(df[feat], df[target])
    correlations.append({
        'Feature': feat,
        'Pearson_r': r,
        'p_value': p,
        'Significant': 'Yes' if p < 0.05 else 'No'
    })

corr_df = pd.DataFrame(correlations)
corr_df = corr_df.sort_values('p_value')

print('单词属性与困难模式占比的Pearson相关分析:')
print(corr_df.to_string(index=False))

### 3.2 Spearman秩相关

Spearman相关系数对非线性单调关系更稳健。

In [ ]:
# Spearman相关分析
spearman_corr = []
for feat in word_features:
    rho, p = stats.spearmanr(df[feat], df[target])
    spearman_corr.append({
        'Feature': feat,
        'Spearman_rho': rho,
        'p_value': p,
        'Significant': 'Yes' if p < 0.05 else 'No'
    })

spearman_df = pd.DataFrame(spearman_corr)
spearman_df = spearman_df.sort_values('p_value')

print('单词属性与困难模式占比的Spearman相关分析:')
print(spearman_df.to_string(index=False))

### 3.3 相关性可视化

**图4说明**：各特征与困难模式占比的相关系数柱状图（含显著性标记）。

In [ ]:
# 图4: 相关系数柱状图
fig, ax = plt.subplots(figsize=FIGSIZE_WIDE)

# 合并Pearson和Spearman结果
corr_df_sorted = corr_df.sort_values('Pearson_r', key=abs, ascending=True)

colors = [COLORS['accent'] if sig == 'Yes' else COLORS['neutral'] 
          for sig in corr_df_sorted['Significant']]

bars = ax.barh(corr_df_sorted['Feature'], corr_df_sorted['Pearson_r'], 
               color=colors, edgecolor='black', alpha=0.8)

ax.axvline(x=0, color='black', linewidth=1)
ax.set_xlabel('Pearson Correlation Coefficient', fontsize=12)
ax.set_ylabel('Word Feature', fontsize=12)

# 添加显著性标记
for i, (feat, row) in enumerate(corr_df_sorted.iterrows()):
    if row['Significant'] == 'Yes':
        ax.annotate('*', xy=(row['Pearson_r'], i), fontsize=14, 
                    ha='left' if row['Pearson_r'] > 0 else 'right')

# 添加图例
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=COLORS['accent'], edgecolor='black', label='Significant (p<0.05)'),
                   Patch(facecolor=COLORS['neutral'], edgecolor='black', label='Not Significant')]
ax.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.savefig('figures/fig4_correlation_barplot.pdf', bbox_inches='tight')
plt.show()
print('图4已保存: figures/fig4_correlation_barplot.pdf')

---
## 4. 多元线性回归分析

In [ ]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# 准备数据
X = df[word_features].copy()
y = df[target].copy()

# 标准化（便于比较系数大小）
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# 添加常数项
X_scaled_const = sm.add_constant(X_scaled)

# 拟合OLS回归
model = sm.OLS(y, X_scaled_const).fit()

print('多元线性回归结果:')
print(model.summary())

In [ ]:
# 提取回归系数和显著性
regression_results = pd.DataFrame({
    'Feature': model.params.index[1:],  # 去除常数项
    'Coefficient': model.params.values[1:],
    'Std_Error': model.bse.values[1:],
    't_value': model.tvalues.values[1:],
    'p_value': model.pvalues.values[1:],
    'Significant': ['Yes' if p < 0.05 else 'No' for p in model.pvalues.values[1:]]
})

regression_results = regression_results.sort_values('p_value')

print('回归系数分析（标准化系数）:')
print(regression_results.to_string(index=False))

print(f'\n模型R-squared: {model.rsquared:.4f}')
print(f'模型Adjusted R-squared: {model.rsquared_adj:.4f}')
print(f'F统计量: {model.fvalue:.4f}')
print(f'F检验p值: {model.f_pvalue:.6f}')

### 4.1 回归系数可视化

**图5说明**：多元线性回归标准化系数及95%置信区间。

In [ ]:
# 图5: 回归系数及置信区间
fig, ax = plt.subplots(figsize=FIGSIZE_WIDE)

# 获取置信区间
conf_int = model.conf_int()
conf_int = conf_int.iloc[1:]  # 去除常数项

# 排序
sorted_idx = regression_results.sort_values('Coefficient', key=abs, ascending=True)['Feature']

y_pos = range(len(sorted_idx))
coefs = [model.params[feat] for feat in sorted_idx]
errors = [(model.params[feat] - conf_int.loc[feat, 0], 
           conf_int.loc[feat, 1] - model.params[feat]) for feat in sorted_idx]
errors = np.array(errors).T

colors = [COLORS['accent'] if model.pvalues[feat] < 0.05 else COLORS['neutral'] 
          for feat in sorted_idx]

ax.barh(y_pos, coefs, xerr=errors, color=colors, edgecolor='black', 
        alpha=0.8, capsize=3)
ax.axvline(x=0, color='black', linewidth=1)
ax.set_yticks(y_pos)
ax.set_yticklabels(sorted_idx)
ax.set_xlabel('Standardized Coefficient (with 95% CI)', fontsize=12)
ax.set_ylabel('Feature', fontsize=12)

# 图例
legend_elements = [Patch(facecolor=COLORS['accent'], edgecolor='black', label='Significant (p<0.05)'),
                   Patch(facecolor=COLORS['neutral'], edgecolor='black', label='Not Significant')]
ax.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.savefig('figures/fig5_regression_coefficients.pdf', bbox_inches='tight')
plt.show()
print('图5已保存: figures/fig5_regression_coefficients.pdf')

---
## 5. 随机森林特征重要性分析

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

# 训练随机森林
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=5)
rf_model.fit(X, y)

# 交叉验证
cv_scores = cross_val_score(rf_model, X, y, cv=5, scoring='r2')

print('随机森林模型:')
print(f'  5折交叉验证R2: {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})')

# 特征重要性
feature_importance = pd.DataFrame({
    'Feature': word_features,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print('\n特征重要性排序:')
print(feature_importance.to_string(index=False))

### 5.1 特征重要性可视化

**图6说明**：随机森林特征重要性排序。

In [ ]:
# 图6: 特征重要性
fig, ax = plt.subplots(figsize=FIGSIZE_NORMAL)

feature_importance_sorted = feature_importance.sort_values('Importance', ascending=True)

bars = ax.barh(feature_importance_sorted['Feature'], 
               feature_importance_sorted['Importance'],
               color=COLORS['primary'], edgecolor='black', alpha=0.8)

ax.set_xlabel('Feature Importance', fontsize=12)
ax.set_ylabel('Feature', fontsize=12)

# 添加数值标签
for bar, val in zip(bars, feature_importance_sorted['Importance']):
    ax.text(bar.get_width() + 0.005, bar.get_y() + bar.get_height()/2,
            f'{val:.3f}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig('figures/fig6_feature_importance.pdf', bbox_inches='tight')
plt.show()
print('图6已保存: figures/fig6_feature_importance.pdf')

---
## 6. 散点图分析（关键特征）

**图7说明**：关键单词属性与困难模式占比的散点图，直观展示变量关系。

In [ ]:
# 图7: 关键特征散点图
key_features = ['num_vowels', 'num_repeated_letters', 'avg_letter_freq', 'min_letter_freq']

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for i, feat in enumerate(key_features):
    ax = axes[i]
    ax.scatter(df[feat], df['hard_mode_ratio'] * 100, 
               alpha=0.5, color=COLORS['primary'], edgecolor='white', s=50)
    
    # 添加回归线
    z = np.polyfit(df[feat], df['hard_mode_ratio'] * 100, 1)
    p = np.poly1d(z)
    x_line = np.linspace(df[feat].min(), df[feat].max(), 100)
    ax.plot(x_line, p(x_line), color=COLORS['secondary'], linewidth=2, linestyle='--')
    
    # 计算相关系数
    r, pval = stats.pearsonr(df[feat], df['hard_mode_ratio'])
    ax.text(0.05, 0.95, f'r = {r:.3f}\np = {pval:.3f}', 
            transform=ax.transAxes, fontsize=10, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    ax.set_xlabel(feat, fontsize=11)
    ax.set_ylabel('Hard Mode Ratio (%)', fontsize=11)

plt.tight_layout()
plt.savefig('figures/fig7_scatter_plots.pdf', bbox_inches='tight')
plt.show()
print('图7已保存: figures/fig7_scatter_plots.pdf')

---
## 7. 统计结论与解释

In [ ]:
# 汇总统计结论
print('='*70)
print('问题二统计分析结论')
print('='*70)

# 1. 相关性分析结论
sig_pearson = corr_df[corr_df['Significant'] == 'Yes']
sig_spearman = spearman_df[spearman_df['Significant'] == 'Yes']

print('\n【1. 相关性分析】')
print(f'  Pearson相关显著的特征数: {len(sig_pearson)} / {len(word_features)}')
print(f'  Spearman相关显著的特征数: {len(sig_spearman)} / {len(word_features)}')

if len(sig_pearson) > 0:
    print(f'  显著特征: {sig_pearson["Feature"].tolist()}')
else:
    print('  无显著相关的特征')

# 2. 回归分析结论
sig_regression = regression_results[regression_results['Significant'] == 'Yes']

print('\n【2. 多元线性回归】')
print(f'  模型R-squared: {model.rsquared:.4f}')
print(f'  F检验p值: {model.f_pvalue:.6f}')
print(f'  显著系数数量: {len(sig_regression)} / {len(word_features)}')

if len(sig_regression) > 0:
    print(f'  显著特征: {sig_regression["Feature"].tolist()}')

# 3. 随机森林结论
print('\n【3. 随机森林特征重要性】')
print(f'  交叉验证R2: {cv_scores.mean():.4f}')
print(f'  最重要特征: {feature_importance.iloc[0]["Feature"]} (importance={feature_importance.iloc[0]["Importance"]:.4f})')

# 4. 总结结论
print('\n【4. 总结结论】')
if model.rsquared < 0.1 and len(sig_pearson) == 0:
    print('  结论: 单词属性对困难模式占比没有显著影响')
    print('  解释: 困难模式选择主要取决于玩家个人偏好，而非单词特征')
elif len(sig_pearson) > 0:
    print('  结论: 存在部分显著相关的单词属性')
    print(f'  显著特征: {sig_pearson["Feature"].tolist()}')
else:
    print('  结论: 单词属性对困难模式占比影响微弱')

print('='*70)

---
## 8. 行为学解释

In [ ]:
# 分析困难模式选择的时间稳定性
print('困难模式占比的时间稳定性分析:')
print(f'  变异系数(CV): {df["hard_mode_ratio"].std()/df["hard_mode_ratio"].mean()*100:.2f}%')

# 按月分组查看
monthly_hard_mode = df.groupby('month')['hard_mode_ratio'].agg(['mean', 'std'])
monthly_hard_mode['cv'] = monthly_hard_mode['std'] / monthly_hard_mode['mean'] * 100

print('\n月度困难模式占比:')
print(monthly_hard_mode.round(4))

### 行为学解释

**为什么单词属性对困难模式占比影响不显著？**

1. **模式选择先于单词出现**：玩家在看到当日单词之前就已决定使用哪种模式
2. **个人偏好主导**：困难模式选择反映的是玩家的「挑战偏好」和「游戏风格」
3. **时间稳定性**：困难模式占比在整年保持相对稳定（CV较低），不随单词变化

---
## 9. 结果汇总

In [ ]:
# 保存分析结果
results_summary = {
    'analysis_type': 'Word Attributes vs Hard Mode Ratio',
    'n_samples': len(df),
    'hard_mode_ratio_mean': df['hard_mode_ratio'].mean(),
    'hard_mode_ratio_std': df['hard_mode_ratio'].std(),
    'regression_r_squared': model.rsquared,
    'regression_f_pvalue': model.f_pvalue,
    'n_significant_features': len(sig_regression),
    'rf_cv_r2_mean': cv_scores.mean(),
    'rf_cv_r2_std': cv_scores.std(),
    'conclusion': 'No significant relationship' if model.rsquared < 0.1 else 'Weak relationship'
}

pd.DataFrame([results_summary]).to_csv('results_summary.csv', index=False)
print('结果已保存至 results_summary.csv')

# 保存相关性分析结果
corr_df.to_csv('correlation_analysis.csv', index=False)
regression_results.to_csv('regression_analysis.csv', index=False)
feature_importance.to_csv('feature_importance.csv', index=False)
print('详细分析结果已保存')

In [ ]:
# 最终结论
print('\n' + '='*70)
print('问题二最终结论')
print('='*70)
print('''
研究问题：单词属性是否影响困难模式玩家占比？

分析方法：
  1. Pearson/Spearman相关分析
  2. 多元线性回归（含显著性检验）
  3. 随机森林特征重要性分析

统计结论：
  - 回归模型R-squared < 0.1，说明单词属性几乎不能解释困难模式占比的变异
  - 大多数特征的相关系数接近0，且不显著
  - 随机森林的预测能力也很低（CV R2接近0）

最终结论：单词属性对困难模式占比没有显著影响

原因解释：
  1. 玩家在开始游戏前就已选择模式，此时尚未看到目标单词
  2. 困难模式选择反映的是玩家的个人偏好和挑战意愿
  3. 困难模式玩家群体相对固定，与每日单词难度无关
''')
print('='*70)

---
## 附录：图片清单

| 编号 | 文件名 | 内容 | 建议插入位置 |
|------|--------|------|-------------|
| 图1 | fig1_hard_mode_distribution.pdf | 困难模式占比分布与时间趋势 | 4.2.1 数据描述 |
| 图2 | fig2_word_features_distribution.pdf | 单词属性特征分布 | 4.2.1 特征描述 |
| 图3 | fig3_correlation_heatmap.pdf | 相关性热力图 | 4.2.2 相关分析 |
| 图4 | fig4_correlation_barplot.pdf | 相关系数柱状图（含显著性） | 4.2.2 相关分析 |
| 图5 | fig5_regression_coefficients.pdf | 回归系数及置信区间 | 4.2.3 回归分析 |
| 图6 | fig6_feature_importance.pdf | 随机森林特征重要性 | 4.2.3 特征重要性 |
| 图7 | fig7_scatter_plots.pdf | 关键特征散点图 | 4.2.2 关系可视化 |